To execute this jupyter notebook, you need to ssh into an analytics machine (I'm using stat1005) and kinit to access HDFS, following the guidelines laid out on wikitech at this page: https://wikitech.wikimedia.org/wiki/Analytics/Systems/Jupyter#Access

Since I will be posting this on publicly on Github, I've made sure to clear all outputs that show individual actor signatures or pageviews. People running this notebook with proper access to WMF resources should be able to run all cells and see intermediate outputs as they like.

In [3]:
import wmfdata
from dataclasses import dataclass
import numpy as np
import random
import math
from IPython.display import Latex

In [9]:
spark = wmfdata.spark.get_session(type='yarn-large')
# spark = wmfdata.spark.get_session()

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [1]:
df = spark.sql("""
SELECT
  pageview_info['page_title'] as page_title,
  page_id,
  pageview_info['project'] as project,
  geocoded_data['country'] as country,
  actor_signature
FROM wmf.pageview_actor
WHERE year = 2021 AND month = 8 AND day = 15 AND hour = 6 AND page_id IS NOT NULL
""").rdd

NameError: name 'spark' is not defined

In [ ]:
df.take(10)

This notebook is going to roughly follow along with the exercises in the two DP tutorials linked within OpenMined's PipelineDP github repo https://github.com/OpenMined/PipelineDP/blob/main/docs/tutorial_1/1_beam_introduction.ipynb, but adapt them to be in Spark rather than Beam.

In [4]:
rdd = df.rdd

In [5]:
# show total number of non-redirect pageviews from August 15 2021 0600 UTC
rdd.count()

25090951

In [6]:
# get distinct number of page ids to see how many pages were viewed
page_id_rdd = rdd.map(lambda x: x.page_id)
page_id_rdd = page_id_rdd.distinct()
page_id_rdd.count()

5234767

In [7]:
# get distinct number of actor signatures to see how many people viewed pages
actor_signature_rdd = rdd.map(lambda x: x.actor_signature)
actor_signature_rdd = actor_signature_rdd.distinct()
actor_signature_rdd.count()

8216201

In [8]:
# get distinct number of countries people viewed pages from
country_rdd = rdd.map(lambda x: x.country)
country_rdd = country_rdd.distinct()
country_rdd.count()

241

In [65]:
# get views per page (no differential privacy)
page_count_rdd = rdd.map(lambda x: (x.page_id, 1))
page_count_rdd = page_count_rdd.reduceByKey(lambda x, y: (x + y))

page_count_rdd.take(25)

[(5929680, 7),
 (44274576, 1),
 (521472, 26271),
 (58800, 56),
 (31767072, 9),
 (8806032, 8),
 (39696, 100),
 (57813216, 1),
 (58551840, 4),
 (3242976, 1),
 (70224, 10),
 (26081616, 1),
 (48488880, 3),
 (256176, 20),
 (101184, 36),
 (12358992, 3),
 (173568, 22),
 (1227696, 701),
 (177840, 369),
 (44486544, 38182),
 (403680, 5),
 (39626736, 20),
 (14078832, 1),
 (49680, 62),
 (24469200, 1)]

In [66]:
# get views per country (no differential privacy)
country_count_rdd = rdd.map(lambda x: (x.country, 1))
country_count_rdd = country_count_rdd.reduceByKey(lambda x, y: (x + y))

country_count_rdd.take(25)

[('Mexico', 279289),
 ('Ukraine', 221744),
 ('Ethiopia', 6278),
 ('Federated States of Micronesia', 89),
 ('Isle of Man', 594),
 ('Martinique', 614),
 ('United Arab Emirates', 58357),
 ('Egypt', 41435),
 ('Djibouti', 425),
 ('Bulgaria', 50488),
 ('Afghanistan', 4971),
 ('Saint Pierre and Miquelon', 5),
 ('Suriname', 333),
 ('Nigeria', 36259),
 ('Gabon', 580),
 ('Cyprus', 20860),
 ('Ghana', 5416),
 ('British Virgin Islands', 61),
 ('American Samoa', 89),
 ('Lithuania', 23774),
 ('Dominica', 84),
 ('Sint Maarten', 49),
 ('Malta', 4317),
 ('Sierra Leone', 363),
 ('Vanuatu', 175)]

In [67]:
# get views per project (no differential privacy)
project_count_rdd = rdd.map(lambda x: (x.project, 1))
project_count_rdd = project_count_rdd.reduceByKey(lambda x, y: (x + y))

project_count_rdd.take(25)

[('uk.wikipedia', 93400),
 ('pl.wikipedia', 282118),
 ('ka.wikipedia', 15851),
 ('ta.wikipedia', 34023),
 ('ht.wikipedia', 1041),
 ('xal.wikipedia', 147),
 ('pa.wikipedia', 2401),
 ('ko.wikinews', 98),
 ('id.wikiquote', 190),
 ('nds.wiktionary', 222),
 ('ne.wiktionary', 202),
 ('ast.wiktionary', 312),
 ('bjn.wikipedia', 314),
 ('st.wiktionary', 25),
 ('csb.wiktionary', 20),
 ('zh.wikiquote', 1031),
 ('ml.wikisource', 788),
 ('nl.wiktionary', 8183),
 ('en.wikinews', 4612),
 ('new.wikipedia', 1091),
 ('lmo.wikipedia', 2708),
 ('zh-classical.wikipedia', 965),
 ('pfl.wikipedia', 141),
 ('pi.wikipedia', 93),
 ('xh.wikipedia', 139)]

In [ ]:
# get views per (project, country, page) (no differential privacy)
tuple_count_rdd = rdd.map(lambda x: ((x.project, x.country, x.page_id), 1))
tuple_count_rdd = tuple_count_rdd.reduceByKey(lambda x, y: (x + y))

tuple_count_rdd.take(25)

In [ ]:
# bound total contributions per actor signature to n
n = 10

# rekey to actor signature
# (actor_signature, pageview)
bounded_views_per_user_rdd = rdd.map(lambda x: (x.actor_signature, [x]))

# randomly get a set of at most `n` pageviews for each actor signature
# (actor_signature, [pageview])
bounded_views_per_user_rdd = bounded_views_per_user_rdd.reduceByKey(lambda x, y: random.sample(x + y, min(len(x) + len(y), n)))

# drop actor signature as key
# ([pageview])
bounded_views_per_user_rdd = bounded_views_per_user_rdd.map(lambda x: x[1])

# unnest lists of pageviews using a flatmap
# (pageview)
bounded_views_per_user_rdd = bounded_views_per_user_rdd.flatMap(lambda x: x)

bounded_views_per_user_rdd.take(25)

In [ ]:
# bound contributions per partition (in this case, per page) and per actor signature

# total contributions (aka sensitivity) = max_per_partition * max_partitions
max_partitions = 5    # say that users can visit at most 5 pages
max_per_partition = 2 # and for each page they can contribute at most 2 pageviews

# rekey to a tuple of (actor signature, page id)
# ((actor_signature, page_id), pageview)
bounded_views_per_partition_rdd = rdd.map(lambda x: ((x.actor_signature, x.page_id), [x]))

# randomly get a set of at most `max_per_partition` pageviews for each (actor signature, page id) tuple
# ((actor_signature, page_id), [pageview]) {max length of max_per_partition}
bounded_views_per_partition_rdd = bounded_views_per_partition_rdd.reduceByKey(lambda x, y: random.sample(x + y, min(len(x) + len(y), max_per_partition)))

# rekey to just actor signature
# (actor_signature, [pageview]) {with redundancies}
bounded_views_per_partition_rdd = bounded_views_per_partition_rdd.map(lambda x: ((x[0][0], x[1])))

# randomly get a set of at most `max_partitions` sets of pageviews for each actor signature
# (actor_signature, [pageview]) {max length of max_per_partition * max_partitions}
bounded_views_per_partition_rdd = bounded_views_per_partition_rdd.reduceByKey(lambda x, y: random.sample(x + y, min(len(x) + len(y), max_partitions)))

# drop actor signature as key
# ([pageview])
bounded_views_per_partition_rdd = bounded_views_per_partition_rdd.map(lambda x: x[1])

# unnest lists of pageviews using a flatmap
# (pageview)
bounded_views_per_partition_rdd = bounded_views_per_partition_rdd.flatMap(lambda x: x)

bounded_views_per_partition_rdd.take(25)

In [33]:
# add laplace noise to a single number
def add_laplace_noise(x, eps, sensitivity):
    scale = sensitivity / eps
    return x + np.random.laplace(scale)

# add laplace noise to a spark rdd
def add_laplace_noise_to_rdd(rdd, eps, max_partitions, max_per_partition):
    eps_per_partition = eps / max_partitions
    sensitivity_per_partition = max_per_partition
    return rdd.map(lambda x: (x[0], add_laplace_noise(x[1], eps_per_partition, sensitivity_per_partition)))

Now we want to calculate a threshold below which we drop data. Given delta (i.e. the probability that we unwittingly release private data), we need to figure out a threshold to drop data beneath such that:

In [3]:
%%latex
\begin{align}
  Pr[1 + laplace(\frac{sensitivity}{\epsilon}) > threshold] <= \delta
\end{align}

<IPython.core.display.Latex object>

Intuition for this step explained in this blog post: https://desfontain.es/privacy/almost-differential-privacy.html


The PDF of the Laplace mechanism is:

In [26]:
%%latex
\begin{align}
    f(x | \mu, b) = \frac{1}{2b} \exp(-\frac{|x - \mu|}{b}).
\end{align}

<IPython.core.display.Latex object>

Since we are always centering our Laplace mechanism on 0, we can drop the 𝜇 argument, and since in this case we are strictly concerned with values of x > 0 (we want to ensure that we rarely *add* enough noise so as to make articles with only 1 pageview go above the threshold), we can also drop the absolute value.
  
That gives us:

In [27]:
%%latex
\begin{align}
    f(x | b) = \frac{1}{2b} \exp(-\frac{x}{b})
\end{align}

<IPython.core.display.Latex object>

where b is the scale of the Laplace noise. In our case, scale = sensitivity / epsilon. Now, for some scale b and a probability delta, we are trying to solve for x such that:

In [32]:
%%latex
\begin{align}
    \delta = \frac{1}{2b} \exp(-\frac{x}{b}) \\
    2b\delta = \exp(-\frac{x}{b}) \\
    \ln(2b\delta) = -\frac{x}{b} \\
    -b \ln(2b\delta) = x
\end{align}

<IPython.core.display.Latex object>

We can set both delta and b (indirectly through sensitivity and epsilon), allowing us to calculate x. We do that below.

In [34]:
def calculate_threshold(delta, eps, max_partitions, max_per_partition):
    eps_per_partition = eps / max_partitions
    sensitivity_per_partition = max_per_partition
    b = sensitivity_per_partition / eps_per_partition
    return -b * math.log(2 * b * delta)

In [92]:
# do bounded DP count

# total contributions (aka sensitivity) = max_per_partition * max_partitions
max_partitions = 5    # say that users can visit at most 5 pages
max_per_partition = 2 # and for each page they can contribute at most 2 pageviews

eps = 1
delta = 5e-8

# rekey to a tuple of (actor signature, page id)
# ((actor_signature, page_id), pageview)
dp_count_rdd = rdd.map(lambda x: ((x.actor_signature, x.page_id), [x]))

# randomly get a set of at most `max_per_partition` pageviews for each (actor signature, page id) tuple
# ((actor_signature, page_id), [pageview]) {max length of max_per_partition}
dp_count_rdd = dp_count_rdd.reduceByKey(lambda x, y: random.sample(x + y, min(len(x) + len(y), max_per_partition)))

# rekey to just actor signature
# (actor_signature, [pageview]) {with redundancies}
dp_count_rdd = dp_count_rdd.map(lambda x: ((x[0][0], x[1])))

# randomly get a set of at most `max_partitions` sets of pageviews for each actor signature
# (actor_signature, [pageview]) {max length of max_per_partition * max_partitions}
dp_count_rdd = dp_count_rdd.reduceByKey(lambda x, y: random.sample(x + y, min(len(x) + len(y), max_partitions)))

# drop actor signature as key
# ([pageview])
dp_count_rdd = dp_count_rdd.map(lambda x: x[1])

# unnest lists of pageviews using a flatmap
# (pageview)
dp_count_rdd = dp_count_rdd.flatMap(lambda x: x)

# now that contributions are bounded, count views per tuple
dp_count_rdd = dp_count_rdd.map(lambda x: ((x.project, x.country, x.page_id, x.page_title), 1))
dp_count_rdd = dp_count_rdd.reduceByKey(lambda x, y: (x + y))

# add laplace noise to counts
dp_count_rdd = add_laplace_noise_to_rdd(dp_count_rdd, eps, max_partitions, max_per_partition)

# filter tuples that have less than `min_number_of_views` views
dp_count_rdd = dp_count_rdd.filter(lambda x: x[1] >= calculate_threshold(delta, eps, max_partitions, max_per_partition))

# round view count to integers for readability
dp_count_rdd = dp_count_rdd.map(lambda x: (x[0], round(x[1], 0)))

dp_count_rdd.takeOrdered(200, key=lambda x: -x[1])

[(('en.wikipedia', 'United States', 15580374, 'Main_Page'), 49280.0),
 (('de.wikipedia', 'Germany', 5248757, 'Wikipedia:Hauptseite'), 24415.0),
 (('ja.wikipedia', 'Japan', 253348, 'メインページ'), 24251.0),
 (('en.wikipedia', 'India', 15580374, 'Main_Page'), 16164.0),
 (('en.wikipedia', 'United Kingdom', 15580374, 'Main_Page'), 13112.0),
 (('fr.wikipedia', 'Russia', 1034876, 'Questionnaire'), 11549.0),
 (('ja.wikipedia', 'Japan', 3093109, 'ジャッキー・ウー'), 11468.0),
 (('fr.wikipedia', 'France', 10635368, 'Wikipédia:Accueil_principal'),
  10308.0),
 (('en.wikipedia', 'India', 3349824, 'Vikram_Batra'), 9991.0),
 (('ja.wikipedia', 'Japan', 2069252, '鍛治舎巧'), 9677.0),
 (('it.wikipedia', 'Italy', 665216, "Gianfranco_D'Angelo"), 9573.0),
 (('ja.wikipedia', 'Japan', 303551, '馬淵史郎'), 8409.0),
 (('en.wikipedia', 'India', 2499568, 'Independence_Day_(India)'), 8218.0),
 (('en.wikipedia', 'Australia', 15580374, 'Main_Page'), 7957.0),
 (('en.wikipedia', 'Iran', 15580374, 'Main_Page'), 7932.0),
 (('ja.wikipedia

In [9]:
from dp_accounting import accountant
from dp_accounting import common

In [10]:
params = common.DifferentialPrivacyParameters(epsilon=0.1, delta=1e-7)

In [ ]:
accountant.get_smallest_laplace_noise(privacy_parameters=params,
                                     num_queries=1,
                                     sensitivity=5)